<a href="https://colab.research.google.com/github/roktavia-cmd/RFM/blob/main/ecommerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/ecommerce_v2.csv')
print ("data berhasil dimuat")

data berhasil dimuat


In [ ]:
print("\nInfromasi Kolom Awal:")
df.info()


Infromasi Kolom Awal:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17049 entries, 0 to 17048
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order_ID                  17049 non-null  object 
 1   Customer_ID               17049 non-null  object 
 2   Date                      17049 non-null  object 
 3   Age                       17049 non-null  int64  
 4   Gender                    17049 non-null  object 
 5   City                      17049 non-null  object 
 6   Product_Category          17049 non-null  object 
 7   Unit_Price                17049 non-null  float64
 8   Quantity                  17049 non-null  int64  
 9   Discount_Amount           17049 non-null  float64
 10  Total_Amount              17049 non-null  float64
 11  Payment_Method            17049 non-null  object 
 12  Device_Type               17049 non-null  object 
 13  Session_Duration_Minutes  17049 non-nu

In [ ]:
#ubah tipe data date
df['Date'] = pd.to_datetime(df['Date'])

#snapshot date(1 hari setelah tanggal transaksi terakhir)
snapshot_date = df['Date'].max() + pd.Timedelta(days=1)
print(f"Tanggal Transaksi Terakhir: {df['Date'].max().date()}")
print(f"Tanggal referensi (Snapshot Date): {snapshot_date.date()}")

#hitung recency, frequency, dan monetary (RFM)
rfm_table = df.groupby('Customer_ID').agg(
    #resensi - hitung hari dari snapshot date ke max date(pembelian terakhir)
    recency = ('Date', lambda x: (snapshot_date - x.max()).days),

    #frekuensi - hitung jumlah order id unik
    frequency = ('Order_ID', 'nunique'),

    #monetary - hitung total pengeluaran (total amount)
    monetary = ('Total_Amount', 'sum')

)



Tanggal Transaksi Terakhir: 2024-03-25
Tanggal referensi (Snapshot Date): 2024-03-26


In [ ]:

#reset index agar customer id menjadi kolom lagi tidak sebagai index
rfm_table = rfm_table.reset_index()

print("-- Tabel RFM Berhasil dibuat --")
print(rfm_table.head())
print("\nInformasi Statistik Dasar Tabel RFM:")
print(rfm_table[['recency', 'frequency', 'monetary']].describe(

))

-- Tabel RFM Berhasil dibuat --
  Customer_ID  recency  frequency  monetary
0  CUST_00001      112          3   2199.63
1  CUST_00002      284          2    809.90
2  CUST_00003       83          2   3030.81
3  CUST_00004       42          1    383.22
4  CUST_00005      279          3   2422.73

Informasi Statistik Dasar Tabel RFM:
           recency    frequency      monetary
count  5000.000000  5000.000000   5000.000000
mean    125.716400     3.409800   4355.810518
std     108.050972     2.181746   5274.546636
min       1.000000     1.000000     14.280000
25%      39.000000     2.000000    889.410000
50%      94.000000     3.000000   2493.925000
75%     185.000000     5.000000   5796.297500
max     450.000000    10.000000  50628.150000


In [ ]:
#menghitung scoring
quintiles = 5
rfm_table['M_Score'] = pd.qcut(rfm_table['monetary'],q=quintiles, labels=False, duplicates='drop') +1
rfm_table['F_Score'] = pd.qcut(rfm_table['frequency'], q=quintiles, labels=False, duplicates='drop') +1
rfm_table['R_Score'] = pd.qcut(-rfm_table['recency'], q=quintiles, labels=False, duplicates='drop') + 1

#membuat kolom rfm score
rfm_table['rfm_score'] = rfm_table['R_Score'].astype(str) + rfm_table['F_Score'].astype(str) + rfm_table['M_Score'].astype(str)


In [ ]:
#fungsi untuk menetapkan segmen berdsarkan skor rfm
def assign_rfm_segment(df) :
  if df['rfm_score'] == '555':
    return 'Champions' #Terbaik
  elif df['rfm_score'] == '444':
    return 'Loyal Customers' #loyalitas tinggi
  elif df['R_Score'] >= 4 and df['F_Score'] >= 4 and df['M_Score'] >= 4:
    return 'Potential Loyalist'
  elif df['R_Score'] >= 4 and df['F_Score'] <= 2:
    return 'New Customer'
  elif df['R_Score'] >= 2 and df['F_Score'] >= 4:
    return 'At Risk' #sering beli tapi lama tidak beli (resioko megnhilang)
  elif df['R_Score'] <= 1 and df['F_Score'] <= 2:
    return 'Lost'  #terburuk: sudah lama, jarang, sedikit belanja
  else:
    return 'Other/Requere Attention' #sefmen lain atau yang memerlukan strategi spesifik


In [ ]:
#membuat kolom baru segmen
rfm_table['Segment'] = rfm_table.apply(assign_rfm_segment, axis=1)

#melihat hasil
print("-- Segmentasi Pelanggan Setelah Dibuat --")
segment_counts = rfm_table['Segment'].value_counts()
print(segment_counts)

#menghitung stastistik rfm persegmen untuk insign
segment_analysis = rfm_table.groupby('Segment')[['recency', 'frequency', 'monetary']].mean().sort_values(by='monetary', ascending=False)
print("-- Analisis Rata-Rata RFM per Segmen --")
print(segment_analysis)

-- Segmentasi Pelanggan Setelah Dibuat --
Segment
Other/Requere Attention    2358
Lost                        946
New Customer                875
Potential Loyalist          450
At Risk                     298
Loyal Customers              73
Name: count, dtype: int64
-- Analisis Rata-Rata RFM per Segmen --
                            recency  frequency      monetary
Segment                                                     
Potential Loyalist        24.020000   7.635556  11811.932156
At Risk                   93.258389   7.110738   7913.601510
Loyal Customers           47.205479   7.054795   5081.399863
Other/Requere Attention  112.065734   3.133164   3907.659406
New Customer              34.692571   2.288000   2961.299291
Lost                     308.593023   1.679704   2039.207104


In [ ]:
rfm_table.to_csv('rfm_analysis_final.csv', index=False)
print('File siap diunduh')

File siap diunduh
